# Population Data Preprocessing

In [13]:
import pandas as pd
import numpy as np

population_df = pd.read_csv("input_data/population_by_country_AUG.2024.csv")

## Data Cleaning and Initial Preprocessing

This cell performs initial data cleaning:
1. Checks for missing values in the dataset.
2. Removes rows with missing values.
3. Checks for and removes duplicate entries in the dataset.

In [14]:
# Check for missing values
print("\nMissing values in population_df dataset:")
print(population_df.isnull().sum())

# Remove rows with missing values
population_df = population_df.dropna()

# Check for duplicates
print("\nDuplicates in population_df dataset:")
print(population_df.duplicated().sum())

# Remove duplicates
population_df = population_df.drop_duplicates()


Missing values in population_df dataset:
Geography     0
Population    0
dtype: int64

Duplicates in population_df dataset:
0


## Identifying Regions and Assigning to Countries

This section identifies regions and assigns them to countries:
1. Identifies rows that represent regions (entries in all caps in the 'Geography' column).
2. Creates a new 'region' column in the dataset.
3. Iterates through the dataset, assigning the appropriate region to each country.

In [15]:
# Identify the rows that represent regions (ALL CAPS entries in 'Geography')
regions = population_df[population_df['Geography'].str.isupper()].copy()

# Create an empty column for regions in the population_df
population_df['region'] = None

# Iterate over the regions and assign the corresponding region to countries
current_region = None
for index, row in population_df.iterrows():
    if row['Geography'].isupper():
        # If the row is a region, set it as the current region
        current_region = row['Geography']
    else:
        # If the row is a country, assign the current region
        population_df.at[index, 'region'] = current_region

## Creating the Final Country Population Dataset

This cell creates the final country population dataset:
1. Removes rows corresponding to regions, keeping only country data.
2. Renames the 'Geography' column to 'country' for clarity.

In [ ]:
# Remove the rows corresponding to regions and keep only the country rows
country_population_df = population_df[~population_df['Geography'].str.isupper()].copy()

# Rename the columns to 'country'
country_population_df = country_population_df.rename(columns={'Geography': 'country'})

# Save the final DataFrame to a CSV file for future use
country_population_df.to_csv('intermediate_data/population_by_country_with_region.csv', index=False)

# Politicians Data Preprocessing

In [16]:
import pandas as pd
import numpy as np
import requests
import json
import pandas as pd
from tqdm import tqdm
from urllib.parse import unquote

politicians_df = pd.read_csv("input_data/politicians_by_country_AUG.2024.csv")

## Data Cleaning and Initial Preprocessing

This cell performs initial data cleaning:
1. Checks for missing values in the dataset.
2. Removes rows with missing values.
3. Checks for and removes duplicate entries in the dataset.

In [17]:
# Check for missing values
print("\nMissing values in population_df dataset:")
print(population_df.isnull().sum())

# Remove rows with missing values
population_df = population_df.dropna()

# Check for duplicates
print("\nDuplicates in population_df dataset:")
print(population_df.duplicated().sum())

# Remove duplicates
population_df = population_df.drop_duplicates()


Missing values in population_df dataset:
Geography      0
Population     0
region        24
dtype: int64

Duplicates in population_df dataset:
0


## Get Predicted Score

### Constants

In [ ]:
# Constants
ORES_API_ENDPOINT = "https://ores.wikimedia.org/v3/scores/enwiki/{revid}/articlequality"
MEDIAWIKI_API_ENDPOINT = "https://en.wikipedia.org/w/api.php"
INPUT_FILE = 'input_data/politicians_by_country_AUG.2024.csv'
OUTPUT_FILE = 'intermediate_data/politicians_with_quality_and_revisions.csv'
ERROR_LOG_FILE = 'intermediate_data/error_log.txt'

REQUEST_HEADER = {
    'User-Agent': "<pgupta1@uw.edu>, University of Washington, MSDS DATA 512 - AUTUMN 2024"
}

## Helper Functions

These functions handle various tasks:
1. Extracting article titles from URLs.
2. Getting the current revision ID for a given article.
3. Requesting ORES quality scores for article revisions.

In [ ]:

def extract_title_from_url(url):
    """Extract the article title from a Wikipedia URL."""
    parts = url.split("/")
    if len(parts) > 4:
        return unquote(parts[4].replace("_", " "))
    return url

def get_current_revision(title):
    """Get the current revision ID for a given Wikipedia article title."""
    params = {
        "action": "query",
        "format": "json",
        "titles": title,
        "prop": "revisions",
        "rvprop": "ids",
        "rvlimit": 1
    }
    
    try:
        response = requests.get(MEDIAWIKI_API_ENDPOINT, params=params, headers=REQUEST_HEADER)
        response.raise_for_status()
        data = response.json()
        page = next(iter(data['query']['pages'].values()))
        return page['revisions'][0]['revid']
    except Exception as e:
        return None

def request_ores_score(rev_id):
    """Get ORES quality prediction for a given article revision."""
    try:
        response = requests.get(ORES_API_ENDPOINT.format(revid=rev_id), headers=REQUEST_HEADER)
        response.raise_for_status()
        data = response.json()
        return data['enwiki']['scores'][str(rev_id)]['articlequality']['score']['prediction']
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except Exception as e:
        print(f"Error getting ORES score for rev_id {rev_id}: {str(e)}")
    return None

### Main Processing Function

This function is the main workhorse of the script:
1. It loads the input data.
2. Processes each article to get its current revision ID and quality score.
3. Handles errors and logs them.
4. Saves progress periodically and outputs the final results.

In [11]:

def process_politicians_data():
    """Process all the politicians data and get quality predictions."""
    df = pd.read_csv(INPUT_FILE)
    
    print("Columns in the CSV file:")
    print(df.columns.tolist())
    
    url_column = 'url' 
    
    if url_column not in df.columns:
        raise ValueError(f"Column '{url_column}' not found in the CSV file.")
    
    df['quality'] = None
    df['revision_id'] = None
    error_count = 0
    error_log = []

    for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing articles"):
        url = row[url_column]
        title = extract_title_from_url(url)
        rev_id = get_current_revision(title)
        
        if rev_id:
            df.at[index, 'revision_id'] = rev_id
            quality = request_ores_score(rev_id)
            if quality:
                df.at[index, 'quality'] = quality
            else:
                error_count += 1
                error_msg = f"Failed to get ORES score for {title} (rev_id: {rev_id})"
                error_log.append(error_msg)
        else:
            error_count += 1
            error_msg = f"Failed to get revision ID for {title}"
            error_log.append(error_msg)
        
        # Save progress every 100 articles
        if (index + 1) % 100 == 0:
            df.to_csv(OUTPUT_FILE, index=False)
            print(f"Progress saved. Processed {index+1} articles.")

    # Calculate and print error rate
    total_articles = len(df)
    error_rate = error_count / total_articles
    print(f"\nError rate: {error_rate:.2%}")

    if error_rate > 0.01:
        print("Error rate is above 1%. Please review the error log.")
    
    # Log errors
    with open(ERROR_LOG_FILE, 'w') as f:
        for error in error_log:
            f.write(f"{error}\n")

    # Save final results
    df.to_csv(OUTPUT_FILE, index=False)
    print(f"\nFinal results saved to {OUTPUT_FILE}")
    print(f"Error log saved to {ERROR_LOG_FILE}")

if __name__ == "__main__":
    process_politicians_data()

Columns in the CSV file:
['name', 'url', 'country']


Processing articles:   0%|          | 0/7155 [00:00<?, ?it/s]

Processing articles:   1%|▏         | 100/7155 [02:01<2:26:12,  1.24s/it]

Progress saved. Processed 100 articles.


Processing articles:   3%|▎         | 200/7155 [04:00<2:44:08,  1.42s/it]

...

Progress saved. Processed 7000 articles.


Processing articles:  99%|█████████▉| 7100/7155 [2:28:08<01:05,  1.18s/it]

Progress saved. Processed 7100 articles.


Processing articles: 100%|██████████| 7155/7155 [2:29:15<00:00,  1.25s/it]


Error rate: 0.17%

Final results saved to politicians_with_quality_and_revisions.csv
Error log saved to error_log.txt
